In [1]:
import numpy as np
import pandas as pd
import scimap as sm
import scanpy as sc
import anndata as ad
import os
import shutil

Running SCIMAP  2.1.3


In [2]:
img_name = "TS-373_IMC21_UB_001"
adata = ad.read_h5ad(f"/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/phenotyping/phenotyping_outputs/uncorrected/standard/{img_name}standard.h5ad")
adata_old = ad.read_h5ad("/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/adatas/old_preprocessed/cells.h5ad")
channel_names = list(adata_old.var_names)
adata.uns['all_markers'] = channel_names

In [15]:
adata.obs

,area,Y_centroid,X_centroid,axis_major_length,axis_minor_length,eccentricity,near_bone,distance_to_bone,CellID,imageid,phenotype
phenotyping_TS-373_IMC26_B_002_1,17,0.823529,28.705882,6.977663,3.108884,0.895258,0.0,119.0,1,phenotyping_TS-373_IMC26_B_002,Neutrophils
phenotyping_TS-373_IMC26_B_002_2,7,0.428571,35.714286,4.155375,1.905471,0.888666,0.0,112.0,2,phenotyping_TS-373_IMC26_B_002,MPO+
phenotyping_TS-373_IMC26_B_002_3,27,1.333333,84.851852,8.065405,4.329692,0.843695,0.0,63.0,3,phenotyping_TS-373_IMC26_B_002,Neutrophils
phenotyping_TS-373_IMC26_B_002_4,18,1.555556,95.944444,5.430024,4.639740,0.519517,0.0,52.0,4,phenotyping_TS-373_IMC26_B_002,Neutrophils
phenotyping_TS-373_IMC26_B_002_5,27,1.666667,125.518519,7.025352,5.074452,0.691574,0.0,22.0,5,phenotyping_TS-373_IMC26_B_002,Plasma Cells/MM cells
...,...,...,...,...,...,...,...,...,...,...,...
phenotyping_TS-373_IMC26_B_002_4987,26,997.538462,416.076923,7.269045,4.735742,0.758653,0.0,301.0,4987,phenotyping_TS-373_IMC26_B_002,Neutrophils
phenotyping_TS-373_IMC26_B_002_4988,23,997.956522,901.956522,8.299189,3.813297,0.888189,0.0,545.0,4988,phenotyping_TS-373_IMC26_B_002,Macrophages/Monocytes
phenotyping_TS-373_IMC26_B_002_4989,21,997.809524,138.000000,6.923115,4.124777,0.803134,0.0,54.0,4989,phenotyping_TS-373_IMC26_B_002,Plasma Cells/MM cells
phenotyping_TS-373_IMC26_B_002_4990,8,998.500000,187.500000,4.472136,2.000000,0.894427,0.0,100.0,4990,phenotyping_TS-373_IMC26_B_002,CD8+Tcells


In [7]:
adata.obs['phenotype'].value_counts()

phenotype
Plasma Cells/MM cells              6431
CD68+                               345
Unknown                             341
CD8+Tcells                          307
Others                              302
MPO+                                296
CD45+                               262
Endothelial cells                   231
HSCs                                124
Neutrophils                          87
CD4+Tcells                           49
Adipocytes                           40
activated Macrophages/Monocytes      15
Macrophages/Monocytes                12
Osteoblasts                          10
Dendritic Cells                       5
Name: count, dtype: int64

In [4]:
adata2 = ad.read_h5ad("/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/adatas/cells_annotated_pp_osteocytes_cleaned_nbh.h5ad")
adata2.uns['all_markers'] = channel_names
adata2 = adata2[adata2.obs['image_ID'] == f'{img_name}.csv']
adata2.obs['Phenotype3'].value_counts()

Phenotype3
Plasma Cells/MM cells              5843
CD8+Tcells                          518
Unknown                             423
Neutrophils                         267
CD4+Tcells                          115
MPO+                                113
Endothelial cells                   104
Adipocytes                           94
CD68+                                89
HSCs                                 40
Osteocyte                            30
activated Macrophages/Monocytes      21
Macrophages/Monocytes                14
Osteoblasts                           6
Dendritic Cells                       4
Osteoclasts                           2
Name: count, dtype: int64

In [3]:
subset = img_name
image_path = f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/img/{subset}.tiff'
mask_path = f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/masks/{subset}.tiff'
sm.pl.image_viewer(image_path = image_path, adata = adata, overlay = 'phenotype', point_color='white', imageid = 'imageid', point_size=5, seg_mask = mask_path)

In [253]:
sm.pl.image_viewer(image_path = image_path, adata = adata2, overlay = 'Phenotype3', point_color='white', imageid = 'imageid', point_size=5, seg_mask = mask_path)

In [254]:
df = pd.read_csv(f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/merged_csv/{img_name}.csv', index_col=0)

In [256]:
adata2

View of AnnData object with n_obs × n_vars = 7683 × 33
    obs: 'Object', 'area', 'Y_centroid', 'X_centroid', 'axis_major_length', 'axis_minor_length', 'eccentricity', 'distance_to_bone', 'Phenotype', 'image_ID', 'disease', 'patient_ID', 'ROI', 'disease2', 'distance_to_bone_corrected', 'Phenotype2', 'Phenotype3', 'cellcharter_CN'
    var: 'name', 'channel', 'deepcell'
    uns: 'all_markers'
    layers: 'arcsinh', 'zscore'

In [257]:
df.rename(columns={'centroid-0': 'y', 'centroid-1': 'x'}, inplace=True)
df['image_id'] = img_name

In [258]:
phenotypes = adata2.obs[['Phenotype3', 'Object']]
phenotypes.reset_index(inplace=True)
phenotypes.drop(columns='index', inplace=True)
phenotypes

/var/folders/ry/jl4ll0zx1l709rh4r7_0cq6r0000gn/T/ipykernel_72675/2691805213.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Phenotype3,Object
0,Unknown,1
1,Neutrophils,2
2,Plasma Cells/MM cells,3
3,Plasma Cells/MM cells,4
4,Adipocytes,5
...,...,...
7678,Unknown,7682
7679,CD8+Tcells,7683
7680,Plasma Cells/MM cells,7684
7681,Unknown,7687


In [259]:
df2 = pd.merge(df, phenotypes, on='Object', how='inner')
df2.rename(columns={'Phenotype3': 'phenotype', 'Object': 'Cell_ID'}, inplace=True)
cell_id_col = df2.pop('Cell_ID')
df2.insert(len(df2.columns) - 1, 'Cell_ID', cell_id_col)
df2

,1,2,3,4,5,CD38,Perilipin,Vimentin,B4GALT1,MPO,...,area,y,x,axis_major_length,axis_minor_length,eccentricity,distance_to_bone,image_id,Cell_ID,phenotype
0,4184.131972,11.773098,36.341659,17.656647,0.000000,1.535025,0.602662,2.511867,0.606347,0.111111,...,9,0.666667,268.555556,4.277663,2.643020,0.786284,90.426766,TS-373_IMC07_UB_001,1,Unknown
1,4207.794367,10.582203,37.439011,17.756799,0.090909,1.569213,1.840551,2.256662,1.070399,20.948459,...,11,0.454545,616.727273,6.410748,1.964940,0.951868,112.071406,TS-373_IMC07_UB_001,2,Neutrophils
2,4185.356477,13.086898,38.698987,16.158176,0.162630,4.218005,0.464382,0.223661,1.220659,0.166685,...,23,0.956522,651.347826,8.879990,3.210956,0.932335,132.653684,TS-373_IMC07_UB_001,3,Plasma Cells/MM cells
3,4331.341797,13.434079,40.373350,21.006117,0.000000,2.823039,1.774032,0.250000,2.044356,0.000000,...,8,0.625000,898.375000,3.969610,2.782480,0.713215,329.887860,TS-373_IMC07_UB_001,4,Plasma Cells/MM cells
4,4174.996594,11.648363,38.974447,17.690310,0.566417,1.660740,2.816500,0.595330,1.867280,0.050000,...,20,0.950000,913.850000,7.731011,3.198667,0.910393,344.548981,TS-373_IMC07_UB_001,5,Adipocytes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7678,4457.073853,14.492965,43.905358,18.781123,1.248100,0.407684,0.250000,1.561770,0.676285,0.000000,...,4,998.750000,308.250000,3.464102,1.414214,0.912871,104.000000,TS-373_IMC07_UB_001,7682,Unknown
7679,4450.526590,11.212891,38.598761,19.863454,0.153004,1.109688,0.935594,1.679860,1.203737,0.090775,...,23,997.913043,360.913043,8.256165,3.698747,0.894034,157.000000,TS-373_IMC07_UB_001,7683,CD8+Tcells
7680,4540.688989,12.261795,46.441156,20.280428,0.241604,2.932276,1.318078,1.530829,1.894356,0.223780,...,20,997.800000,516.800000,6.842132,3.782225,0.833324,313.000000,TS-373_IMC07_UB_001,7684,Plasma Cells/MM cells
7681,4438.805213,14.226270,45.856597,23.863481,0.000000,0.293488,0.099658,0.076923,0.237315,0.000000,...,13,998.153846,947.000000,5.203549,3.076923,0.806443,487.935447,TS-373_IMC07_UB_001,7687,Unknown


In [260]:
df2.to_csv(f'/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/myeloma_IMC/quantification/{img_name}.csv', index=True)
shutil.copy(image_path, f'/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/myeloma_IMC/raw_images/multistack_tiffs/{img_name}.tiff')
shutil.copy(mask_path, f'/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/myeloma_IMC/segmentation/{img_name}.tiff')

'/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/myeloma_IMC/segmentation/TS-373_IMC07_UB_001.tiff'

In [23]:
# Good quality images and phenotypes checked, now creating the combined csv based on csv names
adata = ad.read_h5ad('/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/adatas/cells_annotated_pp_osteocytes_cleaned_nbh_macs.h5ad')


In [28]:
lst = []
for i, csv in enumerate(os.listdir('/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/myeloma_IMC/quantification/')):
    if not (csv.startswith('.') or csv.startswith('myeloma')) and csv.endswith('.csv'):
        #i = pd.read_csv(f'/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/myeloma_IMC/quantification/{csv}')
        df = adata[adata.obs['image_ID'] == csv].to_df()
        df2 = adata[adata.obs['image_ID'] == csv].obs
        i = pd.merge(df, df2, left_index=True, right_index=True)
        i.rename(columns={'Phenotype4': 'phenotype', 'Object': 'CellID', 'X_centroid': 'x', 'Y_centroid': 'y'}, inplace=True)
        i.drop(columns=['Phenotype', 'Phenotype2', 'Phenotype3', 'HLA-DR_counts', 'distance_to_bone'], inplace=True)
        lst.append(i)
comb = pd.concat(lst)
comb

,CD38,Perilipin,Vimentin,B4GALT1,MPO,CathepsinK,ATP5A,RUNX2,HIF1A,CD11b,...,axis_minor_length,eccentricity,image_ID,disease,patient_ID,ROI,disease2,distance_to_bone_corrected,cellcharter_CN,phenotype
Object 1 in TS-373_IMC38_B_002.csv,3.081162,2.445452,4.914248,3.262659,0.998546,1.111457,3.863021,1.563304,1.941053,2.574781,...,2.495303,0.874985,TS-373_IMC38_B_002.csv,MM_BD,IMC38,002,MM_BD,119.816526,adaptive_immune,activated Macrophages/Monocytes
Object 2 in TS-373_IMC38_B_002.csv,1.449686,1.445057,3.864532,1.809677,0.230983,0.228769,2.065104,0.610242,1.205404,1.476939,...,3.662604,0.678103,TS-373_IMC38_B_002.csv,MM_BD,IMC38,002,MM_BD,460.908885,stroma_adipocyte,Plasma Cells/MM cells
Object 3 in TS-373_IMC38_B_002.csv,2.286256,2.080325,4.303116,2.380829,0.659623,0.742665,3.227469,0.850710,1.449479,1.801367,...,3.225859,0.957489,TS-373_IMC38_B_002.csv,MM_BD,IMC38,002,MM_BD,259.138959,stroma_adipocyte,Plasma Cells/MM cells
Object 4 in TS-373_IMC38_B_002.csv,1.698429,2.508642,4.809707,2.022134,0.814226,0.279389,2.740041,0.786641,1.405506,0.985527,...,3.133493,0.901851,TS-373_IMC38_B_002.csv,MM_BD,IMC38,002,MM_BD,73.109507,stroma_adipocyte,Plasma Cells/MM cells
Object 5 in TS-373_IMC38_B_002.csv,3.502561,2.346567,4.019302,2.681798,4.687031,1.981153,3.708386,0.752709,4.973332,3.171923,...,4.212145,0.728936,TS-373_IMC38_B_002.csv,MM_BD,IMC38,002,MM_BD,61.911227,bone_myeloid,Neutrophils
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Object 7682 in TS-373_IMC07_UB_001.csv,0.397160,0.247466,1.228546,0.633130,0.000000,0.000000,0.285756,0.000000,0.000000,0.000000,...,1.414214,0.912871,TS-373_IMC07_UB_001.csv,MM_noBD,IMC07,001,MM_noBD,104.000000,bone,Unknown
Object 7683 in TS-373_IMC07_UB_001.csv,0.956848,0.835091,1.290564,1.018363,0.090651,0.000000,0.649017,0.130068,0.058803,0.762014,...,3.698747,0.894034,TS-373_IMC07_UB_001.csv,MM_noBD,IMC07,001,MM_noBD,157.000000,stroma_adipocyte,CD8+Tcells
Object 7684 in TS-373_IMC07_UB_001.csv,1.796810,1.089425,1.211743,1.395367,0.221954,0.138360,1.473655,0.531174,0.095839,0.945384,...,3.782225,0.833324,TS-373_IMC07_UB_001.csv,MM_noBD,IMC07,001,MM_noBD,313.000000,focal_pc_oxphos,Plasma Cells/MM cells
Object 7687 in TS-373_IMC07_UB_001.csv,0.289430,0.099494,0.076847,0.235142,0.000000,0.000000,0.580869,0.000000,0.000000,0.106578,...,3.076923,0.806443,TS-373_IMC07_UB_001.csv,MM_noBD,IMC07,001,MM_noBD,487.935447,stroma_adipocyte,Unknown


In [29]:
comb.columns

Index(['CD38', 'Perilipin', 'Vimentin', 'B4GALT1', 'MPO', 'CathepsinK',
       'ATP5A', 'RUNX2', 'HIF1A', 'CD11b', 'CD45', 'CS', 'CD11c', 'CD36',
       'CD4', 'CD34', 'CD68', 'IL32', 'IDO', 'CD8', 'GranzymeK', 'PKM2',
       'IRF4', 'GLUT1', 'GranzymeB', 'Ki67', 'CollagenTypeI', 'CD3', 'CPT1A',
       'CD98', 'HLA-DR', 'ST6GAL1', 'CD138', 'CellID', 'area', 'y', 'x',
       'axis_major_length', 'axis_minor_length', 'eccentricity', 'image_ID',
       'disease', 'patient_ID', 'ROI', 'disease2',
       'distance_to_bone_corrected', 'cellcharter_CN', 'phenotype'],
      dtype='object')

In [30]:
comb.to_csv('/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/myeloma_IMC/quantification/myeloma_IMC_combined.csv', index=False)